# Real time IoT Data Processing with Azure Databricks

- **Data Sources: Streaming data from IoT devices.**
- **Ingestion: Azure Event Hubs for capturing real-time data.**
- **Processing: Azure Databricks for stream processing using Structured Streaming.**
- **Storage: Processed data stored Azure Data Lake (Delta Format).**
- **Visualisation: Data visualized using Power BI.**


## Tools and Technologies Used
- Raspberry Pi Azure IoT Online Simulator (https://azure-samples.github.io/raspberry-pi-web-simulator/)
- Azure IoT Hub
- Azure Event Hub
- Azure Databricks (Unity Catalog enabled)
- Azure Data Lake Storage Gen2
- Power BI


## Azure Databricks Configuration

- **Single Node Compute Cluster**: 15.4 LTS (includes Apache Spark 3.5.0, Scala 2.12)
- **Maven Library installed on Compute Cluster**: com.microsoft.azure:azure-eventhubs-spark_2.12:2.3.22

In [0]:
#Importing Libraries
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# Medallion data storage architecture with bronze, silver and gold schemas within 'db_iot_data_2538456515664749' catalog
try:
    spark.sql("create schema db_iot_data_2538456515664749.bronze;")
except:
    print('check if bronze schema already exists')

try:
    spark.sql("create schema db_iot_data_2538456515664749.silver")
except:
    print('check if silver schema already exists')

try:
    spark.sql("create schema db_iot_data_2538456515664749.gold;")
except:
    print('check if gold schema already exists')

check if bronze schema already exists
check if silver schema already exists
check if gold schema already exists


### Bronze Layer

In [0]:
# Set up Azure Event Hub connection config
connectionString = "add connection string here"

eventHubName = "eh-iot-data"

ehConf = {
  'eventhubs.connectionString' : sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString),
  'eventhubs.eventHubName': eventHubName
}

In [0]:
# Read and write data to the bronze schema
# Reading stream: Load data from Azure Event Hub into DataFrame
df = spark.readStream \
    .format("eventhubs") \
    .options(**ehConf) \
    .load()

# Visualize the data for debugging
df.display()

# Writing stream: Persist the streaming data to a Delta table 'db_iot_data_2538456515664749.bronze.weather' in 'append' mode with checkpointing
df.writeStream\
    .option("checkpointLocation", "/mnt/db_iot_data_2538456515664749/bronze/weather")\
    .outputMode("append")\
    .format("delta")\
    .toTable("db_iot_data_2538456515664749.bronze.weather")

org.apache.spark.eventhubs.ConnectionStringBuilder$IllegalConnectionStringFormatException: Connection String cannot be parsed.
	at org.apache.spark.eventhubs.ConnectionStringBuilder.parseConnectionString(ConnectionStringBuilder.scala:319)
	at org.apache.spark.eventhubs.ConnectionStringBuilder.<init>(ConnectionStringBuilder.scala:72)
	at org.apache.spark.eventhubs.ConnectionStringBuilder$.apply(ConnectionStringBuilder.scala:432)
	at org.apache.spark.eventhubs.EventHubsConf.name(EventHubsConf.scala:194)
	at org.apache.spark.sql.eventhubs.EventHubsSource.<init>(EventHubsSource.scala:84)
	at org.apache.spark.sql.eventhubs.EventHubsSourceProvider.createSource(EventHubsSourceProvider.scala:84)
	at org.apache.spark.sql.execution.datasources.DataSource.createSource(DataSource.scala:335)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution$$anonfun$1.$anonfun$applyOrElse$1(MicroBatchExecution.scala:188)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution$$anonfun$1.applyOrElse(MicroBatchExecution.scala:185)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution$$anonfun$1.applyOrElse(MicroBatchExecution.scala:183)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformDownWithPruning$1(TreeNode.scala:505)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(origin.scala:85)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDownWithPruning(TreeNode.scala:505)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.org$apache$spark$sql$catalyst$plans$logical$AnalysisHelper$$super$transformDownWithPruning(LogicalPlan.scala:40)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.transformDownWithPruning(AnalysisHelper.scala:379)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.transformDownWithPruning$(AnalysisHelper.scala:375)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.transformDownWithPruning(LogicalPlan.scala:40)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.transformDownWithPruning(LogicalPlan.scala:40)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformDownWithPruning$3(TreeNode.scala:510)
	at org.apache.spark.sql.catalyst.trees.UnaryLike.mapChildren(TreeNode.scala:1314)
	at org.apache.spark.sql.catalyst.trees.UnaryLike.mapChildren$(TreeNode.scala:1313)
	at org.apache.spark.sql.catalyst.plans.logical.LocalLimit.mapChildren(basicLogicalOperators.scala:2030)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDownWithPruning(TreeNode.scala:510)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.org$apache$spark$sql$catalyst$plans$logical$AnalysisHelper$$super$transformDownWithPruning(LogicalPlan.scala:40)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.transformDownWithPruning(AnalysisHelper.scala:379)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.transformDownWithPruning$(AnalysisHelper.scala:375)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.transformDownWithPruning(LogicalPlan.scala:40)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.transformDownWithPruning(LogicalPlan.scala:40)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformDownWithPruning$3(TreeNode.scala:510)
	at org.apache.spark.sql.catalyst.trees.UnaryLike.mapChildren(TreeNode.scala:1314)
	at org.apache.spark.sql.catalyst.trees.UnaryLike.mapChildren$(TreeNode.scala:1313)
	at org.apache.spark.sql.catalyst.plans.logical.GlobalLimit.mapChildren(basicLogicalOperators.scala:2009)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDownWithPruning(TreeNode.scala:510)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.org$apache$spark$sql$catalyst$plans$logical$AnalysisHelper$$super$transformDownWithPruning(LogicalPlan.scala:40)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.transformDownWithPruning(AnalysisHelper.scala:379)
	at org.apache.spark.sql.ca

### Silver Layer

In [0]:
# Defining the schema for the JSON object
json_schema = StructType([
    StructField("temperature", IntegerType()),
    StructField("humidity", IntegerType()),
    StructField("windSpeed", IntegerType()),
    StructField("windDirection", StringType()),
    StructField("precipitation", IntegerType()),
    StructField("conditions", StringType())
])

In [0]:
# Load streaming data from the 'bronze.weather' Delta table
df = spark.readStream\
    .format("delta")\
    .table("db_iot_data_2538456515664749.bronze.weather")\
    .withColumn("body", col("body").cast("string"))\
    .withColumn("body",from_json(col("body"), json_schema))\
    .select("body.temperature", "body.humidity", "body.windSpeed", "body.windDirection", "body.precipitation", "body.conditions", col("enqueuedTime").alias('timestamp'))

# Visualize the transformed data for debugging
df.display()

# Save the transformed data to the 'silver.weather' Delta table in 'append' mode with checkpointing for data reliability
df.writeStream\
    .option("checkpointLocation", "/mnt/db_iot_data_2538456515664749/silver/weather")\
    .outputMode("append")\
    .format("delta")\
    .toTable("db_iot_data_2538456515664749.silver.weather")

### Gold Layer

In [0]:
# Read from 'silver.weather' apply watermarking and calculate average weather values
df = spark.readStream\
    .format("delta")\
    .table("db_iot_data_2538456515664749.silver.weather")\
    .withWatermark("timestamp", "1 minutes") \
    .groupBy(window("timestamp", "5 minutes")) \
    .agg(avg("temperature").alias('temperature'), avg("humidity").alias('humidity'), avg("windSpeed").alias('windSpeed'), avg("precipitation").alias('precipitation'))\
	.select('window.start', 'window.end', 'temperature', 'humidity', 'windSpeed', 'precipitation')

# Visualize the aggregated data for debugging
df.display()

# Save the aggregated data to the 'gold.weather_summary' Delta table in 'append' mode with checkpointing for data reliability
df.writeStream\
    .option("checkpointLocation", "/mnt/db_iot_data_2538456515664749/gold/weather_summary")\
    .outputMode("append")\
    .format("delta")\
    .toTable("db_iot_data_2538456515664749.gold.weather_summary")

In [0]:
%sql
select * from db_iot_data_2538456515664749.gold.weather_summary